# libraries

In [ ]:
import pandas as pd
import os
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas_profiling
import seaborn as sn 

import json
import requests

from sklearn.model_selection import train_test_split

# import & merge data

In [ ]:
path = os.getcwd()
missing_values = ["n/a", "na", "--"]
data1 = pd.read_csv(path + '/dfJan_Jun_2018.csv',na_values = missing_values)
data2 = pd.read_csv('/Users/haimatias/Desktop/****.out.csv')
data3 = pd.read_csv('/Users/haimatias/Desktop/****.out.csv', encoding='cp1252',error_bad_lines=True)
frames = [data1,data2,data3]
full_data = pd.concat(frames)

# 3D data preprocessing & cleaning

##### data profiling -> observing highly correlated/skewed/cardinality values
##### save to report file

In [ ]:
data_profiling = pandas_profiling.ProfileReport(full_data)
data_profiling.to_file(outputfile='Data_Profile.html')

In [ ]:
full_data['SECURE_3D_DESC'].fillna(0, inplace=True)
non_3D_secure = full_data[full_data['SECURE_3D']==0]
3D_secure = full_data[full_data['SECURE_3D']!=0]

In [ ]:
3D_secure['APPROVED'] = np.where(3D_secure ['APPROVED_AMNT']==0, 0, 1)
non_3D_secure['APPROVED'] = np.where(non_3D_secure['APPROVED_AMNT']==0, 0, 1)

##### missing data , repalce & drop

In [ ]:
3D_secure.isna().sum()
3D_secure['****'].fillna('method', inplace=True)
3D_secure.drop(['*****'],axis=1,inplace =True)

##### value normalization

In [ ]:
3D_secure['****'] = StandardScaler().fit_transform(3D_secure['****'].values.reshape(-1,1)                                                    

##### correlation map

In [ ]:
## Correlation Matrix
sn.set(style="white")

# Compute the correlation matrix
corr = 3D_secure.corr()
corr.head()
# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(18, 15))

# Generate a custom diverging colormap
cmap = sn.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sn.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

# 3DSecure acceptance rate vs lost transaction

In [ ]:
3D_secure['APPROVED_AMNT'].value_counts(normalize=True)

In [ ]:
non_3D_secure['APPROVED_AMNT'].value_counts(normalize=True)

# 3D acceptance rate by CC type

In [ ]:
3D_secure_approved['CARD_SCHEME_DESC'].value_counts(normalize=True)
non_3D_secure_approved['CARD_SCHEME_DESC'].value_counts(normalize=True)

# 3D acceptance rate by currencies

In [ ]:
3D_app_USD = 3D_secure_approved[3D_secure_approved['TRX_CURRENCY']=='USD']
3D_app_GBP = 3D_secure_approved[3D_secure_approved['TRX_CURRENCY']=='GBP']
3D_app_EUR = 3D_secure_approved[3D_secure_approved['TRX_CURRENCY']=='EUR']

In [ ]:
3D_app_USD['APPROVED_AMNT'].value_counts(normalize=True)
3D_app_GBP['APPROVED_AMNT'].value_counts(normalize=True)
3D_app_EUR['APPROVED_AMNT'].value_counts(normalize=True)

# 3D vs regular transaction -avg/med approved amnt by currencies 

In [ ]:
print('###########   EUR   ###########')
print(datanot3dapp_EUR['APPROVED_AMNT'].mean())
print(datanot3dapp_EUR['APPROVED_AMNT'].median())
print('###########   GBP   ###########')
print(datanot3dapp_GBP['APPROVED_AMNT'].mean())
print(datanot3dapp_GBP['APPROVED_AMNT'].median())
print('###########   USD   ###########')
print(datanot3dapp_USD['APPROVED_AMNT'].mean())
print(datanot3dapp_USD['APPROVED_AMNT'].median())

In [ ]:
print('###########   EUR   ###########')
print(3D_app_EUR['APPROVED_AMNT'].mean())
print(3D_app_EUR['APPROVED_AMNT'].median())
print('###########   GBP   ###########')
print(3D_app_GBP['APPROVED_AMNT'].mean())
print(3D_app_GBP['APPROVED_AMNT'].median())
print('###########   USD   ###########')
print(3D_app_USD['APPROVED_AMNT'].mean())
print(3D_app_USD['APPROVED_AMNT'].median())

# acceptance rate by country 

In [ ]:
group_country=3D_secure.groupby(['APPROVED']).BIN_COUNTRY_DESC.value_counts(normalize=True).to_frame()
group_country

# top 3 approval purchase

In [ ]:
3D_secure_approved['GTW_Visa_MCC_Desc'].value_counts().head(3)
non_3D_approved['GTW_Visa_MCC_Desc'].value_counts().head(3)

#  bin-api integration

In [ ]:
bin_list = []
# For each bin number, we access its information through its API
for i in bin_3d['BIN']:
    num = "{}".format(i)
    
    JSONContent = requests.get('https://neutrinoapi.com/bin-lookup?user-id=****' + num).json()
    bin_list.append([i,JSONContent['country'], JSONContent['card-type'], JSONContent['card-brand'],
                         JSONContent['card-category'], JSONContent['issuer']])
                         
data = pd.DataFrame(bin_list)

In [ ]:
data.rename(columns = {0:'BIN',1:'country',2:'card-type',3:'card-brand',4:'card-category',5:'issuer'},inplace=True)

In [ ]:
result = pd.merge(data,bin_3d,on='BIN')#.reindex(bin_3d_10.index)], axis=1

# Prediction approved transaction

In [ ]:
predictors = train.drop(['APPROVED_AMNT'], axis=1)
target = train["APPROVED_AMNT"]
x_train, x_val, y_train, y_val = train_test_split(predictors, target, test_size = 0.22, random_state = 0)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
gbk = GradientBoostingClassifier()
gbk.fit(x_train, y_train)
y_pred = gbk.predict(x_val)
acc_gbk = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_gbk)

In [ ]:
test_pred = pd.DataFrame(y_pred, columns= ['APPROVED_AMNT'])

In [ ]:
test_pred = pd.DataFrame(y_pred, columns= ['APPROVED_AMNT'])

In [ ]:
new_test